In [2]:
# import pandas, matplotlib and numpy
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg, NavigationToolbar2Tk)
from matplotlib.figure import Figure
import numpy as np
import tkinter as tk
from PIL import Image, ImageTk
import Covid_module as covid
import Stopsearch_module as stopsearch

covid.covid()
stopsearch.lockdown_months()
stopsearch.same_months()
    
def show_daily():
    widget_cleared()    # Clear existing frame widgets
    daily_frame.tkraise()
    daily_frame.pack_propagate(False)
    grouped_df_ltla = covid.covid()
    # get dataframe for Hartlepool
    hartlepool_df = grouped_df_ltla.query('areaName in["Hartlepool"]')
    hartlepool_df
    dataframe_hartlepool = hartlepool_df.copy()
    # reset and drop the index column so values are correctly indexed
    dataframe_hartlepool['NO.'] = dataframe_hartlepool.index
    dataframe_hartlepool = dataframe_hartlepool.reset_index(drop=True)
    dataframe_hartlepool
    # get dataframe for stockton
    stockton_df = grouped_df_ltla.query('areaName in["Stockton-on-Tees"]')
    stockton_df
    stockton_on_tees_df = stockton_df.copy()
    dataframe_stockton = stockton_on_tees_df.reset_index()
    dataframe_stockton = dataframe_stockton.drop("index", axis = 1, inplace = False)
    dataframe_stockton['NO.'] = dataframe_stockton.index
    dataframe_stockton = dataframe_stockton.reset_index(drop=True)
    dataframe_stockton
    def comparison_table():
        
        #join the months together to one dataframe
        daily_dataframe = pd.merge(dataframe_hartlepool, dataframe_stockton, how='inner', on = 'date')
        # rename the columns
        daily_dataframe = daily_dataframe.rename(
            columns = {'TotalDailyInfected_x':'Hartlepool', 'TotalDailyInfected_y':'Stockton-on-Tees'}, 
            inplace = False)
        daily_dataframe = daily_dataframe.reindex(columns=['date', 'Hartlepool', 'Stockton-on-Tees'])
        # Export data
        daily_dataframe.to_csv('daily_comparison.csv', encoding='utf-8', index=False)
    # plot graph showing both region infection change
    instruction = tk.Label(daily_frame, text="Graph Showing Daily Infection Change Spread", 
                           font="DIN", bg=bg_color,fg="white").pack()
    export_button = tk.Button(daily_frame, text="Export Data", font="DIN", bg="#28393a", fg="white", 
                              cursor="hand2", activebackground="#badee2", activeforeground="black", 
                              command=lambda:comparison_table()).pack()
    figure = Figure(figsize=(15,5), dpi=100)
    hartlepool_daily = figure.add_subplot(121)
    stockton_daily = figure.add_subplot(122)
    hartlepool_daily.scatter(dataframe_hartlepool['NO.'], dataframe_hartlepool['DailyNewCases'])
    stockton_daily.scatter(dataframe_stockton['NO.'], dataframe_stockton['DailyNewCases'])
    canvas = FigureCanvasTkAgg(figure, daily_frame)
    canvas.get_tk_widget().pack(fill=tk.BOTH, expand = True)
    hartlepool_daily.get_xaxis().set_visible(True)
    hartlepool_daily.set_xlabel('Day Number')
    hartlepool_daily.set_title('Hartlepool')
    hartlepool_daily.set_ylabel('Infected Persons')
    hartlepool_daily.get_xaxis().set_ticks(np.arange(0,240,20))
    hartlepool_daily.set_ylim(-60, 100)
    stockton_daily.get_xaxis().set_visible(True)
    stockton_daily.set_xlabel('Day Number')
    stockton_daily.set_title('Stockton-on-Tees')
    stockton_daily.set_ylabel('Infected Persons')
    stockton_daily.get_xaxis().set_ticks(np.arange(0,240,20))
    stockton_daily.set_ylim(-60, 100)
    back_button = tk.Button(daily_frame, text="Back", font="DIN", bg="#28393a", fg="white", cursor="hand2", 
                            activebackground="#badee2", activeforeground="black", 
                            command=lambda:load_covid()).pack()

def show_weekly():
    widget_cleared()
    weekly_frame.tkraise()
    weekly_frame.pack_propagate(False)
    # calculate infection total for every 7 days
    # for hartlepool
    grouped_df_ltla = covid.covid()
    # get dataframe for Hartlepool
    hartlepool_df = grouped_df_ltla.query('areaName in["Hartlepool"]')
    hartlepool_df
    hartlepool_weekly = hartlepool_df.copy()
    hartlepool_weekly['date'] = pd.to_datetime(hartlepool_weekly['date']) - pd.to_timedelta(7, unit='d')
    # group data by week
    hartlepool_weekly = (hartlepool_weekly.groupby(['areaName', pd.Grouper(key='date', freq='W-MON')])
                         ['TotalDailyInfected'].sum().reset_index().sort_values('date'))
    hartlepool_weekly['NO.'] = hartlepool_weekly.index
    hartlepool_weekly = hartlepool_weekly.reset_index(drop=True)
    hartlepool_weekly
    # weekly change for stockton
    stockton_df = grouped_df_ltla.query('areaName in["Stockton-on-Tees"]')
    stockton_df
    stockton_weekly = stockton_df.copy()
    stockton_weekly['date'] = pd.to_datetime(stockton_weekly['date']) - pd.to_timedelta(7, unit='d')
    stockton_weekly = (stockton_weekly.groupby(['areaName', pd.Grouper(key='date', freq='W-MON')])
                       ['TotalDailyInfected'].sum().reset_index().sort_values('date'))
    stockton_weekly['NO.'] = stockton_weekly.index
    stockton_weekly = stockton_weekly.reset_index(drop=True)
    stockton_weekly
    def comparison_table():
        #join the months together to one dataframe
        week_dataframe = pd.merge(hartlepool_weekly, stockton_weekly, how='inner', on = 'date')
        # rename the columns
        week_dataframe = week_dataframe.rename(
            columns = {'TotalDailyInfected_x':'Hartlepool', 'TotalDailyInfected_y':'Stockton-on-Tees'}, 
            inplace = False)
        week_dataframe = week_dataframe.drop(week_dataframe.columns[[0,  3]], axis=1, inplace=False)
        week_dataframe = week_dataframe.reindex(columns=['date', 'Hartlepool', 'Stockton-on-Tees'])
        # Export data
        week_dataframe.to_csv('weekly_comparison.csv', encoding='utf-8', index=False) 
    # plot graph to compare weekly data for both regions
    instruction = tk.Label(weekly_frame, text="Graph Showing Weekly Infected Persons Total", font="DIN", 
                           bg=bg_color,fg="white").pack()
    export_button = tk.Button(weekly_frame, text="Export Data", font="DIN", bg="#28393a", fg="white", 
                              cursor="hand2", activebackground="#badee2", activeforeground="black", 
                              command=lambda:comparison_table()).pack()
    figure = Figure(figsize=(15,5), dpi=100)
    hartlepool_week = figure.add_subplot(121)
    stockton_week = figure.add_subplot(122)
    hartlepool_week.bar(hartlepool_weekly['NO.'], hartlepool_weekly['TotalDailyInfected'])
    stockton_week.bar(stockton_weekly['NO.'], stockton_weekly['TotalDailyInfected'])
    canvas = FigureCanvasTkAgg(figure, weekly_frame)
    canvas.get_tk_widget().pack(fill=tk.BOTH, expand = True)
    hartlepool_week.get_xaxis().set_visible(True)
    hartlepool_week.set_xlabel('Week Number')
    hartlepool_week.set_title('Hartlepool')
    hartlepool_week.set_ylabel('Infected Persons')
    hartlepool_week.set_ylim(0, 1000)
    stockton_week.get_xaxis().set_visible(True)
    stockton_week.set_xlabel('Week Number')
    stockton_week.set_title('Stockton-on-Tees')
    stockton_week.set_ylabel('Infected Persons')
    stockton_week.set_ylim(0, 1000)
    back_button = tk.Button(weekly_frame, text="Back", font="DIN", bg="#28393a", fg="white", cursor="hand2", 
                                activebackground="#badee2", activeforeground="black", 
                                command=lambda:load_covid()).pack()

def show_monthly():
    widget_cleared()
    monthly_frame.tkraise()
    monthly_frame.pack_propagate(False)
    # for hartlepool
    grouped_df_ltla = covid.covid()
    # get dataframe for Hartlepool
    hartlepool_df = grouped_df_ltla.query('areaName in["Hartlepool"]')
    hartlepool_df
    week_hpool = hartlepool_df.copy()
    week_hpool['date'] = pd.to_datetime(week_hpool['date']) - pd.to_timedelta(7, unit='d')
    month_hartlepool = week_hpool.copy()
    month_hartlepool = (month_hartlepool.groupby(['areaName', pd.Grouper(key='date', freq='M')])
                        ['TotalDailyInfected'].sum().reset_index().sort_values('date'))
    #tag months
    month_hartlepool['Month'] = pd.DatetimeIndex(month_hartlepool['date']).month_name()
    month_hartlepool
    # for stockton
    stockton_df = grouped_df_ltla.query('areaName in["Stockton-on-Tees"]')
    stockton_df
    week_stockton = stockton_df.copy()
    week_stockton['date'] = pd.to_datetime(week_stockton['date']) - pd.to_timedelta(7, unit='d')
    month_stockton = week_stockton.copy()
    month_stockton = (month_stockton.groupby(['areaName', pd.Grouper(key='date', freq='M')])
                      ['TotalDailyInfected'].sum().reset_index().sort_values('date'))
    #tag months into strings
    month_stockton['Month'] = pd.DatetimeIndex(month_stockton['date']).month_name()
    month_stockton
    def comparison_table():
        #join the months together to one dataframe
        monthly_dataframe = pd.merge(month_hartlepool, month_stockton, how='inner', on = 'Month')
        # rename the columns
        monthly_dataframe = monthly_dataframe.rename(columns = {'TotalDailyInfected_x':'Hartlepool', 
                                                     'TotalDailyInfected_y':'Stockton-on-Tees'}, inplace = False)
        monthly_dataframe = monthly_dataframe.drop(monthly_dataframe.columns[[0, 1, 4, 5]], axis=1, inplace=False)
        monthly_dataframe = monthly_dataframe.reindex(columns=['Month', 'Hartlepool', 'Stockton-on-Tees'])
        # Export data
        monthly_dataframe.to_csv('monthly_comparison.csv', encoding='utf-8')
    instruction = tk.Label(monthly_frame, 
                           text="Comparing Monthly Spread of Infected People in LTLA Hartlepool and Stockton", 
                           font="DIN", bg=bg_color,fg="white").pack()
    export_button = tk.Button(monthly_frame, text="Export Data", font="DIN", bg="#28393a", fg="white", 
                              cursor="hand2", activebackground="#badee2", activeforeground="black", 
                              command=lambda:comparison_table()).pack()
    figure = Figure(figsize=(20,5), dpi=100)
    month_plot = figure.add_subplot(111)
    month_plot.plot(month_hartlepool["Month"], month_hartlepool["TotalDailyInfected"], label = "Hartlepool")
    month_plot.plot(month_stockton["Month"], month_stockton["TotalDailyInfected"], label = "Stockton-on-Tees")
    canvas = FigureCanvasTkAgg(figure, monthly_frame)
    canvas.get_tk_widget().pack(fill=tk.BOTH, expand = True)
    month_plot.set_xlabel('Month')
    month_plot.set_ylabel('Infected Persons')
    month_plot.legend()
    back_button = tk.Button(monthly_frame, text="Back", font="DIN", bg="#28393a", fg="white", cursor="hand2", 
                            activebackground="#badee2", activeforeground="black", 
                            command=lambda:load_covid()).pack()


def breakdown():
    widget_cleared()
    breakdown_frame.tkraise()
    breakdown_frame.pack_propagate(False)
    # for hartlepool
    grouped_df_ltla = covid.covid()
    # get dataframe for Hartlepool
    hartlepool_df = grouped_df_ltla.query('areaName in["Hartlepool"]')
    hartlepool_df
    week_hartlepool = hartlepool_df.copy()
    week_hartlepool['date'] = pd.to_datetime(week_hartlepool['date']) - pd.to_timedelta(7, unit='d')
    month_hartlepool = week_hartlepool.copy()
    month_hartlepool = (month_hartlepool.groupby(['areaName', pd.Grouper(key='date', freq='M')])
                        ['TotalDailyInfected'].sum().reset_index().sort_values('date'))
    #tag months
    month_hartlepool['Month'] = pd.DatetimeIndex(month_hartlepool['date']).month_name()
    month_hartlepool
    # October had the highest daily infected people while July had the lowest
    # for stockton
    stockton_df = grouped_df_ltla.query('areaName in["Stockton-on-Tees"]')
    stockton_df
    week_stockton = stockton_df.copy()
    week_stockton['date'] = pd.to_datetime(week_stockton['date']) - pd.to_timedelta(7, unit='d')
    month_stockton = week_stockton.copy()
    month_stockton = (month_stockton.groupby(['areaName', pd.Grouper(key='date', freq='M')])
                      ['TotalDailyInfected'].sum().reset_index().sort_values('date'))
    #tag months into strings
    month_stockton['Month'] = pd.DatetimeIndex(month_stockton['date']).month_name()
    month_stockton
    # Plot graph to show distribution
    instruction = tk.Label(breakdown_frame, text="Monthly Spread of Infected People in LTLA Hartlepool and Stockton", 
                           font="DIN", bg=bg_color,fg="white").pack()
    figure = Figure(figsize=(30,5), dpi=100, constrained_layout=True)
    hartlepool_month = figure.add_subplot(121)
    stockton_month = figure.add_subplot(122)
    hartlepool_month.plot(month_hartlepool["Month"], month_hartlepool["TotalDailyInfected"])
    stockton_month.plot(month_stockton["Month"], month_stockton["TotalDailyInfected"])
    canvas = FigureCanvasTkAgg(figure, breakdown_frame)
    canvas.get_tk_widget().pack(fill=tk.BOTH, expand = True)
    hartlepool_month.get_xaxis().set_visible(True)
    hartlepool_month.set_xlabel('Month')
    hartlepool_month.set_title('Hartlepool')
    hartlepool_month.set_ylim(0, 3000)
    hartlepool_month.set_ylabel('Infected Persons')
    stockton_month.get_xaxis().set_visible(True)
    stockton_month.set_xlabel('Month')
    stockton_month.set_title('Stockton-on-Tees')
    stockton_month.set_ylim(0, 3000)
    stockton_month.set_ylabel('Infected Persons')
    back_button = tk.Button(breakdown_frame, text="Back", font="DIN", bg="#28393a", fg="white", cursor="hand2", 
                            activebackground="#badee2", activeforeground="black", 
                            command=lambda:load_covid()).pack(pady=20)

# Stop and Search Analysis

    

def outcome():
    widget_cleared()
    outcome_frame.tkraise()
    outcome_frame.pack_propagate(False)
    # get data of people based on outcome.
    lockdown2020_df_mod = stopsearch.lockdown_months()
    stop_search_lockdown = lockdown2020_df_mod.groupby(["outcome"])["involved_person"].count()
    stop_search_lockdown = stop_search_lockdown.reset_index().drop([0],axis = 0, inplace = False)
    
    def comparison_table():
        # Export data
        stop_search_lockdown.to_csv('outcome_data.csv', encoding='utf-8', index=False)
    
    # plot graph
    instruction = tk.Label(outcome_frame, text="Outcome of Stop and Search by Cleveland Police", font="DIN", 
                           bg=bg_color,fg="white").pack()
    export_button = tk.Button(outcome_frame, text="Export Data", font="DIN", bg="#28393a", fg="white", 
                              cursor="hand2", activebackground="#badee2", activeforeground="black", 
                              command=lambda:comparison_table()).pack()
    figure = Figure(figsize=(20,5), dpi=100, constrained_layout=True)
    searchoutcome = figure.add_subplot(111)
    searchoutcome.barh(stop_search_lockdown["outcome"], stop_search_lockdown["involved_person"])
    canvas = FigureCanvasTkAgg(figure, outcome_frame)
    canvas.get_tk_widget().pack(fill=tk.BOTH, expand = True)
    searchoutcome.get_xaxis().set_visible(True)
    searchoutcome.get_yaxis().set_visible(True)
    searchoutcome.set_xlabel('Outcome')
    searchoutcome.set_ylabel('Number of Persons Involved')    
    back_button = tk.Button(outcome_frame, text="Back", font="DIN", bg="#28393a", fg="white", cursor="hand2", 
                            activebackground="#badee2", activeforeground="black", 
                            command=lambda:load_stopsearch()).pack()

def Arrests_gender():
    widget_cleared()
    gender_frame.tkraise()
    gender_frame.pack_propagate(False)
    lockdown2020_df_mod = stopsearch.lockdown_months()
    # get data for arrests
    lockdown_arrested = lockdown2020_df_mod.query('outcome in["Arrest"]')
    # gender count
    gender_count_lockdown = lockdown_arrested.groupby(["gender"])["outcome"].count().reset_index()
    gender_count_lockdown
    
    def comparison_table():
        # Export data
        gender_count_lockdown.to_csv('gender_count_data.csv', encoding='utf-8', index=False)

    # plot graphs to show distribution of arrests in lockdown months by Gender
    instruction = tk.Label(gender_frame, text="Gender Distribution of Stop and Search Arrests by Cleveland Police", 
                           font="DIN", bg=bg_color,fg="white").pack()
    export_button = tk.Button(gender_frame, text="Export Data", font="DIN", bg="#28393a", fg="white", 
                              cursor="hand2", activebackground="#badee2", activeforeground="black", 
                              command=lambda:comparison_table()).pack()
    figure = Figure(figsize=(10,5), dpi=100)
    genderarrest = figure.add_subplot(111)
    genderarrest.pie(gender_count_lockdown["outcome"], labels = gender_count_lockdown["gender"], 
                     autopct = '%1.1f%%')
    canvas = FigureCanvasTkAgg(figure, gender_frame)
    canvas.get_tk_widget().pack(fill=tk.BOTH, expand = True)
    back_button = tk.Button(gender_frame, text="Back", font="DIN", bg="#28393a", fg="white", cursor="hand2", 
                            activebackground="#badee2", activeforeground="black", 
                            command=lambda:load_stopsearch()).pack()

def Arrests_age():
    widget_cleared()
    arrests_frame.tkraise()
    arrests_frame.pack_propagate(False)
    lockdown2020_df_mod = stopsearch.lockdown_months()
    # get data for arrests
    lockdown_arrested = lockdown2020_df_mod.query('outcome in["Arrest"]')
    # age count
    age_count_lockdown = lockdown_arrested.groupby(["age_range"])["outcome"].count().reset_index()
    age_count_lockdown
    
    def comparison_table():
        # Export data
        age_count_lockdown.to_csv('age_count_data.csv', encoding='utf-8', index=False)

    # plot graph
    instruction = tk.Label(arrests_frame, text="Age Distribution of Stop and Search Arrests by Cleveland Police", 
                           font="DIN", bg=bg_color,fg="white").pack()
    export_button = tk.Button(arrests_frame, text="Export Data", font="DIN", bg="#28393a", fg="white", 
                              cursor="hand2", activebackground="#badee2", activeforeground="black", 
                              command=lambda:comparison_table()).pack()
    figure = Figure(figsize=(10,5), dpi=100)
    agearrests = figure.add_subplot(111)
    agearrests.bar(age_count_lockdown["age_range"], age_count_lockdown["outcome"])
    canvas = FigureCanvasTkAgg(figure, arrests_frame)
    canvas.get_tk_widget().pack(fill=tk.BOTH, expand = True)
    agearrests.get_xaxis().set_visible(True)
    agearrests.get_yaxis().set_visible(True)
    agearrests.set_xlabel('Age')
    agearrests.set_ylabel('Number of Persons Involved')
    back_button = tk.Button(arrests_frame, text="Back", font="DIN", bg="#28393a", fg="white", cursor="hand2", 
                            activebackground="#badee2", activeforeground="black", 
                            command=lambda:load_stopsearch()).pack()

def object_searched():
    widget_cleared()
    objects_frame.tkraise()
    objects_frame.pack_propagate(False)
    lockdown2022_df_mod = stopsearch.same_months()
    # Group data according to search object 
    stop_search_lockdown22 = lockdown2022_df_mod.groupby(["object_of_search"])["involved_person"].count()
    stop_search_2022_df = stop_search_lockdown22.reset_index().drop([0],axis = 0, inplace = False)
    stop_search_2022_df
    
    def comparison_table():
        # Export data
        stop_search_2022_df.to_csv('objects_searched_data.csv', encoding='utf-8', index=False)
    
    # plot graph
    instruction = tk.Label(objects_frame, text="Stop and Search Information based on Objects Searched", 
                           font="DIN", bg=bg_color,fg="white").pack()
    export_button = tk.Button(objects_frame, text="Export Data", font="DIN", bg="#28393a", fg="white", 
                              cursor="hand2", activebackground="#badee2", activeforeground="black", 
                              command=lambda:comparison_table()).pack()
    figure = Figure(figsize=(10,5), dpi=100, constrained_layout=True)
    search_object = figure.add_subplot(111)
    search_object.barh(stop_search_2022_df['object_of_search'], stop_search_2022_df['involved_person'])
    canvas = FigureCanvasTkAgg(figure, objects_frame)
    canvas.get_tk_widget().pack(fill=tk.BOTH, expand = True)
    search_object.get_xaxis().set_visible(True)
    search_object.get_yaxis().set_visible(True)
    search_object.set_xlabel('Search Object')
    search_object.set_ylabel('Number of Persons Involved')                      
    back_button = tk.Button(objects_frame, text="Back", font="DIN", bg="#28393a", fg="white", cursor="hand2", 
                            activebackground="#badee2", activeforeground="black", 
                            command=lambda:load_stopsearch()).pack()

def comparison():
    widget_cleared()
    comparison_frame.tkraise()
    comparison_frame.pack_propagate(False)
    # compare age count data in 2020 and 2022
    lockdown2020_df_mod = stopsearch.lockdown_months()
    lockdown2022_df_mod = stopsearch.same_months()
    # get data for arrests
    lockdown_arrested = lockdown2020_df_mod.query('outcome in["Arrest"]')
    lockdown2022_arrested = lockdown2022_df_mod.query('outcome in["Arrest"]')
    # age count
    age_count_lockdown = lockdown_arrested.groupby(["age_range"])["outcome"].count().reset_index()
    age_count22 = lockdown2022_arrested.groupby(["age_range"])["outcome"].count().reset_index()
    #join the months together to one dataframe
    comparison_dataframe = pd.merge(age_count_lockdown, age_count22, how='inner', on = 'age_range')
    # rename the columns
    comparison_dataframe = comparison_dataframe.rename(columns = {'outcome_x':'Lockdown', 'outcome_y':'2022'}, 
                                                       inplace = False)
    comparison_dataframe
    
    def comparison_table():
        # Export data
        data_table = comparison_dataframe.reindex(columns=['age_range', 'Lockdown', '2022'])
        data_table.to_csv('age_arrests_comparison.csv', encoding='utf-8')
    # plot graph
    instruction = tk.Label(comparison_frame, text="Comparing Age Group Arrests in 2020 lockdown Months and 2022", 
                           font="DIN", bg=bg_color,fg="white").pack()
    export_button = tk.Button(comparison_frame, text="Export Data", font="DIN", bg="#28393a", fg="white", 
                              cursor="hand2", activebackground="#badee2", activeforeground="black", 
                              command=lambda:comparison_table()).pack()
    figure = Figure(figsize=(10,5), dpi=100)
    comparing_age = figure.add_subplot(111)
    comparing_age.plot(comparison_dataframe['age_range'], comparison_dataframe['Lockdown'], label = "Lockdown")
    comparing_age.plot(comparison_dataframe['age_range'], comparison_dataframe['2022'], label = "2022")
    canvas = FigureCanvasTkAgg(figure, comparison_frame)
    canvas.get_tk_widget().pack(fill=tk.BOTH, expand = True)
    comparing_age.legend()
    comparing_age.get_xaxis().set_visible(True)
    comparing_age.get_yaxis().set_visible(True)
    comparing_age.set_xlabel('Age Group')
    comparing_age.set_ylabel('Number of Persons Involved')
    back_button = tk.Button(comparison_frame, text="Back", font="DIN", bg="#28393a", fg="white", cursor="hand2", 
                            activebackground="#badee2", activeforeground="black", 
                            command=lambda:load_stopsearch()).pack()

#default background colour
bg_color = "#3d6466"

# Clear existing widget
def clear_widgets(frame):
    for widget in frame.winfo_children():
        widget.destroy()
# to prevent aggregation and repetition of code
def widget_cleared():
    clear_widgets(parent_frame)
    clear_widgets(stopsearch_frame)
    clear_widgets(covid_frame)
    clear_widgets(objects_frame)
    clear_widgets(comparison_frame)
    clear_widgets(arrests_frame)
    clear_widgets(gender_frame)
    clear_widgets(outcome_frame)
    clear_widgets(daily_frame)
    clear_widgets(weekly_frame)
    clear_widgets(monthly_frame)
    clear_widgets(breakdown_frame)
    
# Landing Page
def load_parent():
        clear_widgets(stopsearch_frame)
        clear_widgets(covid_frame)
        parent_frame.tkraise()
        parent_frame.pack_propagate(False)
        # insert logo
        TU = Image.open('TUlogoblack.png')
        TU = ImageTk.PhotoImage(TU)
        logo_widget = tk.Label(parent_frame, image=TU, bg=bg_color)
        logo_widget.image = TU
        logo_widget.pack()
        # instructions
        tk.Label(parent_frame, text="Hello! What Analysis will you be visualising today?!", font="DIN", 
                 bg=bg_color,fg="white").pack()
        # Buttons
        tk.Button
        covid_button = tk.Button(parent_frame, text="CoronaVirus", font="DIN", bg="#28393a", fg="white", 
                                 cursor="hand2", activebackground="#badee2", activeforeground="black", 
                                 command=lambda:load_covid()).pack(pady=20)
        stopsearch_button = tk.Button(parent_frame, text="Stop & Search", font="DIN", bg="#28393a", fg="white", 
                                      cursor="hand2", activebackground="#badee2", activeforeground="black", 
                                      command=lambda:load_stopsearch()).pack(pady=20)
        quit_button = tk.Button(parent_frame, text="Quit", font="DIN", bg="#28393a", fg="white", cursor="hand2", 
                                activebackground="#badee2", activeforeground="black", 
                                command=root.destroy).pack(pady=20)

def load_stopsearch():
    clear_widgets(parent_frame)
    stopsearch_frame.tkraise()
    # insert logo
    CP = Image.open('clevelandpolice.jpg')
    CP = ImageTk.PhotoImage(CP)
    logo_widget = tk.Label(stopsearch_frame, image=CP, bg=bg_color)
    logo_widget.image = CP
    logo_widget.pack(pady=20)
    
    back_button = tk.Button(stopsearch_frame, text="Back to Homepage", font="DIN", bg="#28393a", fg="white", 
                            cursor="hand2", activebackground="#badee2", activeforeground="black", 
                            command=lambda:load_parent()).pack(pady=10)
    tk.Label(stopsearch_frame, text="Select analysis to view", font="DIN", bg=bg_color,fg="white").pack(pady=15)
    # Buttons
    tk.Button
    R1 = tk.Radiobutton(stopsearch_frame, text="stop and search outcomes in Lockdown months (April, May, June 2020)", 
                        font=("DIN", 10), bg="#28393a", fg="white", cursor="hand2", activebackground="#badee2", 
                        activeforeground="black", command=lambda:outcome()).pack(pady=10)
    R2 = tk.Radiobutton(stopsearch_frame, text="Arrests in Lockdown months (April, May, June 2020) by Gender", 
                        font=("DIN", 10), bg="#28393a", fg="white", cursor="hand2", activebackground="#badee2", 
                        activeforeground="black", command=lambda:Arrests_gender()).pack(pady=10)
    R3 = tk.Radiobutton(stopsearch_frame, text="Arrests in Lockdown months (April, May, June 2020) by Age Group", 
                        font=("DIN", 10), bg="#28393a", fg="white", cursor="hand2", activebackground="#badee2", 
                        activeforeground="black", command=lambda:Arrests_age()).pack(pady=10)
    R4 = tk.Radiobutton(stopsearch_frame, text="objects searched during in the same lockdown months in 2022", 
                        font=("DIN", 10), bg="#28393a", fg="white", cursor="hand2", activebackground="#badee2", 
                        activeforeground="black", command=lambda:object_searched()).pack(pady=10)
    R5 = tk.Radiobutton(stopsearch_frame, text="comparing arrests in lockdown months 2020 and 2022 by Age Group", 
                        font=("DIN", 10), bg="#28393a", fg="white", cursor="hand2", activebackground="#badee2", 
                        activeforeground="black", command=lambda:comparison()).pack(pady=10)
    quit_button = tk.Button(stopsearch_frame, text="Quit", font="DIN", bg="#28393a", fg="white", cursor="hand2", 
                            activebackground="#badee2", activeforeground="black", 
                            command=root.destroy).pack(pady=15)
    
def load_covid():
    clear_widgets(parent_frame)
    covid_frame.tkraise()
    # insert logo
    NHS = Image.open('NHS.jpg')
    NHS = ImageTk.PhotoImage(NHS)
    logo_widget = tk.Label(covid_frame, image=NHS, bg=bg_color)
    logo_widget.image = NHS
    logo_widget.pack(pady=20) 
    back_button = tk.Button(covid_frame, text="Back to Homepage", font="DIN", bg="#28393a", fg="white", 
                            cursor="hand2", activebackground="#badee2", activeforeground="black", 
                            command=lambda:load_parent()).pack(pady=10)
    tk.Label(covid_frame, text="Select analysis to view", font="DIN", bg=bg_color,fg="white").pack(pady=15)
    # Buttons
    tk.Button
    C1 = tk.Radiobutton(covid_frame, text="daily change in infection in LTLA :Stockton & Hartlepool", 
                        font=("DIN", 10), bg="#28393a", fg="white", cursor="hand2", activebackground="#badee2", 
                        activeforeground="black", command=lambda:show_daily()).pack(pady=10)
    C2 = tk.Radiobutton(covid_frame, text="total number of people infected weekly in Stockton & Hartlepool", 
                        font=("DIN", 10), bg="#28393a", fg="white", cursor="hand2", activebackground="#badee2", 
                        activeforeground="black", command=lambda:show_weekly()).pack(pady=10)
    C3 = tk.Radiobutton(covid_frame, text="breakdown of infection by months in both areas", font=("DIN", 10), 
                        bg="#28393a", fg="white", cursor="hand2", activebackground="#badee2", 
                        activeforeground="black", command=lambda:breakdown()).pack(pady=10)
    C4 = tk.Radiobutton(covid_frame, text="Comparing Monthly Infection in Both Areas", font=("DIN", 10), 
                        bg="#28393a", fg="white", cursor="hand2", activebackground="#badee2", 
                        activeforeground="black", command=lambda:show_monthly()).pack(pady=10)
    quit_button = tk.Button(covid_frame, text="Quit", font="DIN", bg="#28393a", fg="white", cursor="hand2", 
                            activebackground="#badee2", activeforeground="black", 
                            command=root.destroy).pack(pady=15)

# initialize app
root = tk.Tk()
# give window a title
root.title("Data Analysis Program")
# let windownopen at centre of the screen # root.eval("tk::Placewindow . centre")
# create  parent frame widget
parent_frame = tk.Frame(root, width=1200, height=600, bg=bg_color)
covid_frame = tk.Frame(root, bg=bg_color)
stopsearch_frame = tk.Frame(root, bg=bg_color)
comparison_frame = tk.Frame(root, bg=bg_color)
objects_frame = tk.Frame(root, bg=bg_color)
arrests_frame = tk.Frame(root, bg=bg_color)
gender_frame = tk.Frame(root, bg=bg_color)
outcome_frame = tk.Frame(root, bg=bg_color)
daily_frame = tk.Frame(root, bg=bg_color)
weekly_frame = tk.Frame(root, bg=bg_color)
breakdown_frame = tk.Frame(root, bg=bg_color)
monthly_frame = tk.Frame(root, bg=bg_color)

for frame in (parent_frame, covid_frame, daily_frame, weekly_frame, monthly_frame, breakdown_frame, 
              stopsearch_frame, outcome_frame, gender_frame, arrests_frame, objects_frame, comparison_frame):
    frame.grid(row=0, column=0, sticky="nesw")

load_parent()



# run app
root.mainloop()